In [1]:
%pip install pyaudio
%pip install soundfile

     -------------------------------------- 164.1/164.1 kB 5.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:

import pyaudio
import wave
import numpy as np
def pyaudio_acquisition(RECORD_SECONDS,RATE):
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    CHUNK = 1024
    audio = pyaudio.PyAudio()
     
    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)
    print ("recording...")
    #frames = []
     
    
    buffer = stream.read(RATE * RECORD_SECONDS)
    mydata = np.frombuffer(buffer,dtype = "int16")
    
    
    #for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
     #   data = stream.read(CHUNK)
      #  frames.append(np.fromstring(data, dtype=float))
        #frames.extend(np.frombuffer(data, dtype = float))
    print ("finished recording")
    
    #mydata = np.hstack(frames)
    #mydata = np.hstack(frames)
    
     
    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()
    save = input("Vil du lagre dataene som en .wav fil [Y/n]?  ")
    if save == "Y" or save == "y":
       WAVE_OUTPUT_FILENAME = input("Skriv filnavn på formen filename.wav ")   
       waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
       waveFile.setnchannels(CHANNELS)
       waveFile.setsampwidth(audio.get_sample_size(FORMAT))
       waveFile.setframerate(RATE)
       waveFile.writeframes(b''.join(mydata))
       waveFile.close()
    return mydata
#pyaudio_acquisition(5, 44100)*


In [4]:


import matplotlib.pyplot as plt
import numpy as np
def stykkevisFFT(t,fs,data,fmin):
    """
    Algoritmen som numpy bruker for FFT foretrekker arrays med log_2 lengde.
    For enkelthetens skyld, fortsetter vi med det.
    Grunnen til at vi ofte bruker FFT_length//2 er fordi vi ønsker bare å plotte 
den reelle delen av signalet,
    og halvparten av arrayet som FFT returnerer er den komplekskonjugerte.
    Stykkevis FFT tar Fourier transformasjon på et tidsinterval.
    Vi deler opp total duration inn i intervaller med lengde
    (duration * samplingfreq)/2^x. x er et predefinert heltall.
    Parameters
    ----------
        Aqcuisition time
    fs : Int
        Sampling frequency.
    data : Array
           Sound data aqcuisition.
    fmin : Int
        Minimal frequency to include.
    Returns
    -------
    tw : Array
        Et array med midten av hvert tidsintervall.
    fw : Frekvensen fra hvert tidsintervall.
    n : Int
         ̈Lengden på hvert tidsintervall.
    FFT_freq : Array
        Frekvens aksen i Fourier transformasjonen.
    power : Array
        Kvadrert frekvensamplitude fra Fourier Transformasjon.
    """
    wp2 = 14
    FFT_length = 2**wp2
    #hvert tidsintervall vil få disse punktene å utføre FFT på.
    n = int(len(t)/FFT_length)
    #gjør om t til lengde 2**wp2, dette vil gjøre at man mister et neglisjerbart
    #antall datapunkter.
    t = t[:n*FFT_length].reshape(n,FFT_length)
    y = data[:n*FFT_length].reshape(n,FFT_length)
    tw = t[:,FFT_length//2]    #tw er tiden midt i alle intervallene
    fw = np.zeros(n)
    FFT_freq = fs/2*np.linspace(0,1,FFT_length//2)  #frekvenser til plotting av FFT
    power = np.zeros((n,FFT_length//2,))            # til plotting av 
#absoluttverdien av signalet.
    fmin_index = np.min(np.where(FFT_freq>fmin))
    #Fourier transformerer hvert intervall med datapunkter
    for i in range(n):
        FFT = np.fft.fft(y[i,:],FFT_length)
        #tar bare med FFT'en til frekvenser som er større enn fmin
        #og tar ikke med siste halvdel, siden vi ikke er interessert i 
#komplekskonjugerte
        power[i,:] = np.abs(FFT[:FFT_length//2])**2
        #Vi ønsker å hente ut indeksen til den maksimale amplituden per intervall
        max_amp_indx = np.argmax(np.abs(FFT[fmin_index:FFT_length//2])**2)
        fw[i] = FFT_freq[max_amp_indx+fmin_index-1]
    return tw, fw, n, FFT_freq, power


In [ ]:
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
#from sound_acquisition_pyaudio import pyaudio_acquisition

inputs = input("Har du en lydfil fra før? [Y/n] ")

if inputs == "Y" or inputs == "y":
    filename = input("Skriv inn filnavn på formen filnavn.wav ")
    fmin = int(input("Hva er minimumsfrekvensen? "))
    mydata, samplerate = sf.read(filename)
    duration = int(len(mydata)/samplerate)

if inputs == "N" or inputs == "n":
    temp = input("Skriv inn samplerate [Hz] varighet [s] og fmin [Hz] ").split()
    samplerate = int(temp[0])
    duration = int(temp[1])
    fmin = int(temp[2])

    """For pyaudio unhash linjen under."""
    mydata = pyaudio_acquisition(duration, samplerate)
    """For sounddevice unhash linjen under."""
    #mydata = sound_data_acquisition(duration, samplerate).transpose().reshape(-1)
    """
    Dersom du bruker sounddevice så vil mydata har shapen (1,n),
    for at fourier transformasjonen skal gå riktig for seg,
    er vi nødt til å først transponere den til (n,1), deretter reshape
    den slik at den får formen (n,).
    """

t = np.linspace(0, duration, samplerate*duration)

try:
    print("{} antall samples er registrert." .format(len(mydata)))
except:
    print("\n Har du husket å unhashe mydata linjen?")


tw, fw, n, FFT_freq, power = stykkevisFFT(t,samplerate,mydata,fmin)


fmin_idx = np.argwhere(FFT_freq > fmin)

power = power[:,np.min(fmin_idx):]
FFT_freq = FFT_freq[np.min(fmin_idx):]



plt.style.use("seaborn")
plt.subplot(1,3,1)
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [A.U]")
plt.plot(t,mydata)


plt.subplot(1,3,2)
plt.plot(tw,fw,"o-")
plt.xlabel("Time [s]")
plt.ylabel("Frequency [Hz]")


#plotter bare Fourier for første tidsintervall.
i = 0
plt.subplot(1,3,3)
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude  [A.U]")
plt.plot(FFT_freq,power[i])
plt.savefig("FFT_test.png")
plt.show()


Har du en lydfil fra før? [Y/n] n
Skriv inn samplerate [Hz] varighet [s] og fmin [Hz] 10000 5 100
recording...
finished recording
Vil du lagre dataene som en .wav fil [Y/n]?  y
